In [52]:

import numpy as np
import math
from matplotlib.pylab import mpl
from MyUtils import *
import torch
from Model import CNN
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F #非线性函数在的地方

In [53]:
mpl.rcParams['font.sans-serif'] = ['SimHei']   #显示中文
mpl.rcParams['axes.unicode_minus']=False       #显示负号

获取数据

In [54]:
pos_files = ["u001_w001", "u001_w002", "u001_w003"]
neg_files = ["u002_w001", "u002_w002", "u002_w003"]
files = zip(pos_files, neg_files)


for index, (pos_file, neg_file) in enumerate(files):
    if index == 0:
        features, labels = CNN_row_dataTarget(pos_file, neg_file,sample_size=100)
        all_features = features
        all_labels = labels


    else:
        features, labels = CNN_row_dataTarget(pos_file, neg_file,sample_size=100)
        all_features = np.concatenate((all_features, features), axis=0)
        all_labels = np.concatenate((all_labels, labels), axis=0)

features = all_features
labels = all_labels
print(labels.shape)
features = features.astype(np.float32)
print(features.shape, labels.shape)

(319, 3, 2)
(322, 3, 2)
(529, 3, 2)
(322, 3, 2)
(159, 3, 2)
(322, 3, 2)
pos_features (1973, 3, 2), pos_labels is (1973,)
all_features shape is (6468, 3, 2), neg_labels is (4495,)
特征维度为：(6468, 3, 2),标签维度为：(6468,)
(282, 3, 2)
(285, 3, 2)
(466, 3, 2)
(285, 3, 2)
(141, 3, 2)
(285, 3, 2)
pos_features (1744, 3, 2), pos_labels is (1744,)
all_features shape is (7380, 3, 2), neg_labels is (5636,)
特征维度为：(7380, 3, 2),标签维度为：(7380,)
(302, 3, 2)
(305, 3, 2)
(499, 3, 2)
(305, 3, 2)
(151, 3, 2)
(305, 3, 2)
pos_features (1867, 3, 2), pos_labels is (1867,)
all_features shape is (6357, 3, 2), neg_labels is (4490,)
特征维度为：(6357, 3, 2),标签维度为：(6357,)
(20205,)
(20205, 3, 2) (20205,)


In [55]:
model = CNN()
batch_size = 15
loss_func = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(),lr=0.001)

In [56]:
features ,labels= torch.Tensor(features[:,:,0]),torch.Tensor(labels)
print(features.shape,labels.shape)
features, labels = features.reshape(batch_size,-1,3),labels.reshape(batch_size,-1)
print(features.shape,labels.shape)
xTrain, xTest, yTrain, yTest = train_test_split(features,labels,test_size=0.3,random_state=222)

torch.Size([20205, 3]) torch.Size([20205])
torch.Size([15, 1347, 3]) torch.Size([15, 1347])


In [57]:
loss_count = []
for epoch in range(20):
    for i,(x,y) in enumerate(zip(xTrain,yTrain)):
        model.zero_grad()
        batch_x = x # torch.Size([128, 1, 28, 28])
        batch_y = y # torch.Size([128])
        # 获取最后输出
        out = model.forward(batch_x) # torch.Size([128,10])
        # 获取损失
        loss = loss_func(out,batch_y)
        # 使用优化器优化损失
        opt.zero_grad()  # 清空上一步残余更新参数值
        loss.backward() # 误差反向传播，计算参数更新值
        opt.step() # 将参数更新值施加到net的parmeters上
        if i%20 == 0:
            loss_count.append(loss)
            print('{}:\t'.format(i), loss.item())
            torch.save(model,r'C:\Users\liev\Desktop\myproject\yin_test\log_CNN')
        if i % 100 == 0:
            for a,b in (xTest,yTest):
                test_x = a
                test_y = b
                out = model(test_x)
                # print('test_out:\t',torch.max(out,1)[1])
                # print('test_y:\t',test_y)
                accuracy = torch.max(out,1)[1].numpy() == test_y.numpy()
                print('accuracy:\t',accuracy.mean())
                break


plt.figure('PyTorch_CNN_Loss')
plt.plot(loss_count,label='Loss')
plt.legend()
plt.show()


RuntimeError: Expected 4-dimensional input for 4-dimensional weight[64, 3, 3, 3], but got 2-dimensional input of size [1347, 3] instead